In [ ]:
# Imports
import os 
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = 'true'

In [20]:
from django.forms import model_to_dict
from flourish_child.models import TbReferalAdol as TbCRF
from flourish_prn.models import TbReferalAdol as TbPRN
from edc_metadata.constants import REQUIRED, KEYED
from edc_appointment.constants import COMPLETE_APPT
from flourish_child.models import Appointment as ChildAppointment
from django.db import IntegrityError
    
visit_code = '2100A'

tb_referral_objs = TbPRN.objects.all()

for obj in tb_referral_objs:
    instance_dict = model_to_dict(obj)
    
    del instance_dict['action_identifier']
    del instance_dict['parent_tracking_identifier']
    del instance_dict['related_tracking_identifier']
    del instance_dict['subject_identifier']
    del instance_dict['tracking_identifier']
    
    try:
        child_visit = ChildVisit.objects.get(visit_code=visit_code,
                                             visit_code_sequence = 0,
                                             subject_identifier = obj.subject_identifier)
        
    except ChildVisit.DoesNotExist:
        pass
    else:
        
        try:
            TbCRF.objects.get(
                child_visit=child_visit,
            )
        except TbCRF.DoesNotExist:
            
            TbCRF.objects.create(
                child_visit=child_visit,
                **instance_dict
            )
            
            # Delete metadata so they can be auto created
            CrfMetadata.objects.filter(
                subject_identifier = child_visit.subject_identifier,
                visit_code = child_visit.visit_code,
                visit_code_sequence = child_visit.visit_code_sequence,
            ).delete()
        
            # Update the metadata
            child_visit.save()
            
            print(f'{child_visit.subject_identifier} : Created')
            
        else:
            print(f'{child_visit.subject_identifier} : Already Exist')


        

    



B142-040990838-0-10 : Already Exist
B142-040990705-1-10 : Already Exist
B142-040991059-2-10 : Already Exist
